In [1]:
import json
import os
import getpass
from typing import Dict, Any
from langchain_community.llms import Ollama
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from experimental.base import ExperimentalAPIChain

In [2]:
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

In [3]:
ENV_VARS = (
    {"name": "GOOGLE_API_KEY", "required": True, "force_reset": False, "hide": True},
    {"name": "VAULT_URL", "required": False, "force_reset": False, "hide": False},
    {"name": "VAULT_TOKEN", "required": False, "force_reset": False, "hide": True},
)
for var in ENV_VARS:
    if var["required"] and (var["name"] not in os.environ or var["force_reset"]):
        os.environ[var["name"]] = getpass.getpass(f"{var['name'].replace('_', ' ').title()}: ")
    present = " ✅" if var["name"] in os.environ and len(os.environ[var["name"]]) > 0 else " ❌"
    value = f" {os.environ[var['name']]}" if var["name"] in os.environ and not var["hide"] else ""
    print(f"{var['name']}:{value}{present}")

GOOGLE_API_KEY: ✅
VAULT_URL: http://46.105.247.55:8200 ✅
VAULT_TOKEN: ❌


In [4]:
llm_kwargs: Dict[str, Any] = {
    "model": "llama3:instruct", # Model name
    "temperature": 0,
    "num_ctx": 2048, # Context size,
    "base_url": os.getenv("OLLAMA_HOST", "http://localhost:11434"),
    "callback_manager": CallbackManager([StreamingStdOutCallbackHandler()]),
}

llm = Ollama(**llm_kwargs)

In [5]:
llm_kwargs: Dict[str, Any] = {
    "model": "gemini-1.5-flash-latest",
    "temperature": 0,
    "callbacks": [StreamingStdOutCallbackHandler()],
}

llm = ChatGoogleGenerativeAI(**llm_kwargs)

In [6]:
# Example of processed API docs from https://api.xcloud.coralio.fr/openapi.json

_api_docs = {'base_url': 'https://api.xcloud.coralio.fr/',
 'endpoints': [
        {'/api/chat': {'post': {'tags': ['Chat'
                    ],
     'summary': 'Create Chat',
     'description': 'Creates a chat',
     'operationId': 'create_chat_api_chat_post',
     'requestBody': {'required': True,
      'content': {'application/json': {'schema': {'$ref': {'properties': {'user_id': {'anyOf': [
                                                    {'type': 'string'
                                                    },
                                                    {'type': 'string', 'format': 'uuid'
                                                    }
                                                ],
            'title': 'User Id'
                                            },
           'type': {'type': 'string', 'enum': ['dm', 'gc'
                                                ], 'title': 'Type'
                                            },
           'name': {'anyOf': [
                                                    {'type': 'string'
                                                    },
                                                    {'type': 'null'
                                                    }
                                                ],
            'title': 'Name',
            'default': ''
                                            },
           'participants': {'items': {'anyOf': [
                                                        {'type': 'string'
                                                        },
                                                        {'type': 'string', 'format': 'uuid'
                                                        }
                                                    ]
                                                },
            'type': 'array',
            'title': 'Participants'
                                            }
                                        },
          'type': 'object',
          'required': ['user_id', 'type', 'participants'
                                        ],
          'title': 'ChatCreate'
                                    }
                                }
                            }
                        }
                    },
     'responses': {'201': {'description': 'Successful Response',
       'content': {'application/json': {'schema': {}
                                }
                            }
                        },
      '422': {'description': 'Validation Error',
       'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'
                                    }
                                }
                            }
                        }
                    },
     'apiUrl': 'https://api.xcloud.coralio.fr/api/chat'},
    'get': {'tags': ['Chat'
                        ],
     'summary': 'List Chats',
     'operationId': 'list_chats_api_chat_get',
     'parameters': [
                            {'name': 'user_id',
       'in': 'query',
       'required': True,
       'schema': {'type': 'string', 'title': 'User Id'
                                }
                            },
                            {'name': 'q',
       'in': 'query',
       'required': False,
       'schema': {'type': 'string', 'default': '', 'title': 'Q'
                                }
                            },
                            {'name': 'type',
       'in': 'query',
       'required': False,
       'schema': {'enum': ['dm', 'gc'
                                    ], 'type': 'string', 'title': 'Type'
                                }
                            },
                            {'name': 'limit',
       'in': 'query',
       'required': False,
       'schema': {'type': 'integer',
        'maximum': 25,
        'minimum': 1,
        'default': 10,
        'title': 'Limit'
                                }
                            },
                            {'name': 'offset',
       'in': 'query',
       'required': False,
       'schema': {'type': 'integer',
        'minimum': 0,
        'default': 0,
        'title': 'Offset'
                                }
                            }
                        ],
     'responses': {'200': {'description': 'Successful Response',
       'content': {'application/json': {'schema': {}
                                    }
                                }
                            },
      '422': {'description': 'Validation Error',
       'content': {'application/json': {'schema': {'$ref': '#/components/schemas/HTTPValidationError'
                                        }
                                    }
                                }
                            }
                        },
     'apiUrl': 'https://api.xcloud.coralio.fr/api/chat'}}}]}
api_docs = json.dumps(_api_docs, indent=2)

In [7]:
api_url_template = """
Given the following API Documentation: {api_docs}
Your task is to construct the most efficient API URL to answer 
the user's question, ensuring the 
call is optimized to include only necessary information.
If a parameter is not required and does not have a
default value, do NOT include it in the API URL, unless you think
it's relevant to the user's question.
You MUST extract the API URL, request METHOD and generate the BODY data in JSON format
according to the user question if necessary.
The parameters' names and BODY data keys MUST be obtained from the provided context.
Do NOT make up parameters' names.
The BODY data can be an empty JSON.
Output the API URL, METHOD and BODY. Join them with `|`. DO NOT GIVE ANY EXPLANATION.

Question: {question}

Output:
"""
api_url_prompt = PromptTemplate(input_variables=['api_docs', 'question'],
                                template=api_url_template)

api_response_template = """
With the following official API Documentation: {api_docs} 
and the specific user question: {question} in mind,
and given this API URL: {api_url} for querying, here is the 
response from the API: {api_response}. 
Do NOT include technical details like response format.
Do NOT include any thoughts or internal processes.
You MUST provide a clear, relevant and concise answer.

Response:
"""
api_response_prompt = PromptTemplate(input_variables=['api_docs', 
                                                      'question', 
                                                      'api_url',
                                                      'api_response'],
                                     template=api_response_template)

In [8]:
# Instantiate the new PowerfulAPIChain class

chain = ExperimentalAPIChain.from_llm_and_api_docs(
    llm,
    api_docs=api_docs,
    headers=None,
    verbose=True,
    limit_to_domains=["https://api.xcloud.coralio.fr/"],
    allowed_http_methods=["get", "post"],
)

In [9]:
# Example user request that will prompt the LLM to make an API POST request

participants = ['0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1',
                'ff0d95a2-69ca-41ca-9c91-064259adb7c4']
user_input = (
    f"""Create a chat for the user 'ff0d95a2-69ca-41ca-9c91-064259adb7c4' and participants {participants}.
    It's a group chat called 'devops team'
    """
)
response = chain.invoke(user_input)



> Entering new ExperimentalAPIChain chain...

Request info: {
    "api_url": "https://api.xcloud.coralio.fr/api/chat",
    "request_method": "POST",
    "request_body": {
        "user_id": "ff0d95a2-69ca-41ca-9c91-064259adb7c4",
        "type": "gc",
        "name": "devops team",
        "participants": [
            "0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1",
            "ff0d95a2-69ca-41ca-9c91-064259adb7c4"
        ]
    }
}
API URL: https://api.xcloud.coralio.fr/api/chat
Request method: POST
Request body: {
    "api_url": "https://api.xcloud.coralio.fr/api/chat",
    "request_method": "POST",
    "request_body": {
        "user_id": "ff0d95a2-69ca-41ca-9c91-064259adb7c4",
        "type": "gc",
        "name": "devops team",
        "participants": [
            "0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1",
            "ff0d95a2-69ca-41ca-9c91-064259adb7c4"
        ]
    }
}
Status: 201 Created
Response: {"id":"4edb3527-cf92-498f-ab44-ec558555787b","type":"gc","user_id":"ff0d95a2-69ca-41c

In [10]:
print(
    f"Human: {response['question']}\n"
    f"Assistant: {response['output']}"
)

Human: Create a chat for the user 'ff0d95a2-69ca-41ca-9c91-064259adb7c4' and participants ['0c6d43a9-3bf6-48a0-bf2e-5ab719f009f1', 'ff0d95a2-69ca-41ca-9c91-064259adb7c4'].
    It's a group chat called 'devops team'
    
Assistant: A group chat called 'devops team' was successfully created. 

